In [4]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple tensorflow==2.2.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 175.7 MB 1.1 MB/s eta 0:00:014
     |████████████████████████████████| 42 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 132 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 15.1 MB 5.7 MB/s eta 0:00:011
     |████████████████████████████████| 1.0 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 781 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 7.0 MB/s  eta 0:00:01
     |████████████████████████████████| 288 kB

     |████████████████████████████████| 77 kB 4.9 MB/s eta 0:00:011
Using legacy 'setup.py install' for wrapt, since package 'wheel' is not installed.
Using legacy 'setup.py install' for termcolor, since package 'wheel' is not installed.
    Running setup.py install for wrapt ... done
    Running setup.py install for termcolor ... done
You should consider upgrading via the '/Users/wizhang/.pyenv/versions/3.8.2/bin/python -m pip install --upgrade pip' command.


In [69]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

2.2.3


In [70]:
users = ['清淡,25,ANY', '重口,25,ANY', '清淡,50,中餐', '清淡,100,ANY']
foods = ['桃源春卷', '三圣面', '大食代', '高级茶餐厅', '海底捞', '山西手工面']
features = ['清淡','重口','辣椒','健康','减肥餐','我爱面食','我爱米饭','特色','本地','网红','价格低','价格中','价格高']

num_users = len(users)
num_foods = len(foods)
num_features = len(features)
num_recommdations = 3

In [71]:
# user reates foods
users_foods = tf.constant(
[
        [4, 3, 3, 0, 2, 0],
        [0, 10, 4, 0, 6, 8],
        [10, 3, 3, 0, 3, 2],
        [10, 9, 0, 10, 0, 2]
], dtype = tf.float32
)

# movie has features
foods_features = tf.constant([
        [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0],
        [0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0],
        [1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1],
        [0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1],
        [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0]
], dtype = tf.float32)


In [72]:
# Mattrix: the user rated features
users_features = tf.matmul(users_foods, foods_features)
users_features

<tf.Tensor: shape=(4, 13), dtype=float32, numpy=
array([[ 7.,  8.,  8.,  4.,  4., 10.,  3.,  9.,  3.,  9.,  3.,  7.,  2.],
       [12., 28., 28.,  0.,  0., 22.,  4., 24., 10., 16., 18.,  4.,  6.],
       [15., 11., 11., 10., 10., 18.,  3., 18.,  3., 16.,  5., 13.,  3.],
       [22., 11., 11., 20., 20., 21., 10., 31.,  9., 29., 11., 10., 10.]],
      dtype=float32)>

In [73]:
# scale up
users_features = users_features / tf.reduce_sum(users_features, axis=1, keepdims=True)
users_features

<tf.Tensor: shape=(4, 13), dtype=float32, numpy=
array([[0.09090909, 0.1038961 , 0.1038961 , 0.05194805, 0.05194805,
        0.12987013, 0.03896104, 0.11688311, 0.03896104, 0.11688311,
        0.03896104, 0.09090909, 0.02597403],
       [0.06976745, 0.1627907 , 0.1627907 , 0.        , 0.        ,
        0.12790698, 0.02325581, 0.13953489, 0.05813954, 0.09302326,
        0.10465116, 0.02325581, 0.03488372],
       [0.11029412, 0.08088236, 0.08088236, 0.07352941, 0.07352941,
        0.13235295, 0.02205882, 0.13235295, 0.02205882, 0.11764706,
        0.03676471, 0.09558824, 0.02205882],
       [0.10232558, 0.05116279, 0.05116279, 0.09302326, 0.09302326,
        0.09767442, 0.04651163, 0.14418605, 0.04186046, 0.13488372,
        0.05116279, 0.04651163, 0.04651163]], dtype=float32)>

In [74]:
# Rank the user rated features that each of the users like the most so far
# nn = neural network
top_users_features = tf.nn.top_k(users_features, num_features)[1]
top_users_features # indexes of which movie which user like the most

<tf.Tensor: shape=(4, 13), dtype=int32, numpy=
array([[ 5,  7,  9,  1,  2,  0, 11,  3,  4,  6,  8, 10, 12],
       [ 1,  2,  7,  5, 10,  9,  0,  8, 12,  6, 11,  3,  4],
       [ 5,  7,  9,  0, 11,  1,  2,  3,  4, 10,  6,  8, 12],
       [ 7,  9,  0,  5,  3,  4,  1,  2, 10,  6, 11, 12,  8]], dtype=int32)>

In [75]:
for i in range(num_users):
    feature_names = [features[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users[i], feature_names))


清淡,25,ANY: ['我爱面食', '特色', '网红', '重口', '辣椒', '清淡', '价格中', '健康', '减肥餐', '我爱米饭', '本地', '价格低', '价格高']
重口,25,ANY: ['重口', '辣椒', '特色', '我爱面食', '价格低', '网红', '清淡', '本地', '价格高', '我爱米饭', '价格中', '健康', '减肥餐']
清淡,50,中餐: ['我爱面食', '特色', '网红', '清淡', '价格中', '重口', '辣椒', '健康', '减肥餐', '价格低', '我爱米饭', '本地', '价格高']
清淡,100,ANY: ['特色', '网红', '清淡', '我爱面食', '健康', '减肥餐', '重口', '辣椒', '价格低', '我爱米饭', '价格中', '价格高', '本地']


In [76]:
users_ratings = tf.matmul(users_features, tf.transpose(foods_features))
users_ratings

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.6493506 , 0.64935064, 0.5584416 , 0.4935065 , 0.4675325 ,
        0.5844156 ],
       [0.45348838, 0.84883726, 0.5697675 , 0.3604651 , 0.5930233 ,
        0.76744187],
       [0.73529416, 0.6029412 , 0.52205884, 0.55147064, 0.43382356,
        0.5735295 ],
       [0.71162796, 0.572093  , 0.39534882, 0.6604651 , 0.42790696,
        0.4976744 ]], dtype=float32)>

In [77]:
top_foods = tf.nn.top_k(users_ratings, num_recommdations)[1]
top_foods

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[1, 0, 5],
       [1, 5, 4],
       [0, 1, 5],
       [0, 3, 1]], dtype=int32)>

In [78]:
for i in range(num_users):
    movie_names = [foods[index] for index in top_foods[i]]
    print('{}: {}'.format(users[i], movie_names))

清淡,25,ANY: ['三圣面', '桃源春卷', '山西手工面']
重口,25,ANY: ['三圣面', '山西手工面', '海底捞']
清淡,50,中餐: ['桃源春卷', '三圣面', '山西手工面']
清淡,100,ANY: ['桃源春卷', '高级茶餐厅', '三圣面']


In [79]:
users_ratings_new = tf.where(
    tf.equal(users_foods, tf.zeros_like(users_foods)),
    users_ratings,
    tf.zeros_like(tf.cast(users_foods, tf.float32))
)
users_ratings_new

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.4935065 , 0.        ,
        0.5844156 ],
       [0.45348838, 0.        , 0.        , 0.3604651 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.55147064, 0.        ,
        0.        ],
       [0.        , 0.        , 0.39534882, 0.        , 0.42790696,
        0.        ]], dtype=float32)>

In [80]:
top_foods = tf.nn.top_k(users_ratings_new, num_recommdations)[1]
top_foods

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[5, 3, 0],
       [0, 3, 1],
       [3, 0, 1],
       [4, 2, 0]], dtype=int32)>

In [81]:
for i in range(num_users):
    movie_names = [foods[index] for index in top_foods[i]]
    print('{}: {}'.format(users[i], movie_names))

清淡,25,ANY: ['山西手工面', '高级茶餐厅', '桃源春卷']
重口,25,ANY: ['桃源春卷', '高级茶餐厅', '三圣面']
清淡,50,中餐: ['高级茶餐厅', '桃源春卷', '三圣面']
清淡,100,ANY: ['海底捞', '大食代', '桃源春卷']
